In [1]:
import pandas as pd
import urllib3 as urllib
import json
import IPython.display



In [2]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [3]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [4]:
#Saving a copy of source data 
ltc_data_json = json.dumps(ltc_data)

f = open("Source_data/IL_" + reporting_date + "LTC_data_Source.json","w")
f.write(ltc_data_json)
f.close()

In [5]:
df = pd.DataFrame(ltc_data['FacilityValues'])
df['reporting_date'] = reporting_date
df['% deaths'] = df['deaths'] / df['confirmed_cases']
df['num of facilities'] = 1 

In [6]:
df.to_csv('Reporting_data/IL_' + reporting_date + '_LTC_data_.csv', index = False)

In [7]:
a = df.sum()
print ('Date: %s' % reporting_date)
print ('Cases: %d' % a.confirmed_cases)
print ('Deaths:  %d' % a.deaths)

Date: 2020-06-05
Cases: 18841
Deaths:  3084


In [8]:
df_county = df.groupby(by=['County']).sum()
df_county['% deaths'] = (df_county['deaths'] / df_county['confirmed_cases'])
df_county.sort_values('deaths', ascending=False)

,confirmed_cases,deaths,% deaths,num of facilities
County,,,,
Cook,10932,1798,0.164471,267
DuPage,1845,321,0.173984,58
Lake,1390,202,0.145324,41
Will,747,143,0.191432,28
Kane,773,125,0.161708,28
St. Clair,417,63,0.151079,13
McHenry,333,60,0.180180,10
Madison,211,55,0.260664,7
Winnebago,197,37,0.187817,17


In [9]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O', sort='-x'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2

alt.HConcatChart(...)

In [10]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2 

alt.HConcatChart(...)

In [11]:
df.sort_values(by=['deaths'], ascending=False).to_csv('IL_LTC_Covid.csv')

In [12]:
df['deaths'].sum()

3084

In [14]:
df.head()

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths,num of facilities
0,Boone,Park Place of Belvidere,5,0,Open,2020-06-05,0.000000,1
1,Boone,Symphony Northwoods,87,13,Open,2020-06-05,0.149425,1
2,Champaign,Reflections Memory Care,8,0,Open,2020-06-05,0.000000,1
3,Champaign,University Rehab Center,5,0,Open,2020-06-05,0.000000,1
4,Christian,Villas of Holly Brook,2,0,Open,2020-06-05,0.000000,1


In [15]:
df.sum()

County               BooneBooneChampaignChampaignChristianClintonCl...
FacilityName         Park Place of BelvidereSymphony NorthwoodsRefl...
confirmed_cases                                                  18841
deaths                                                            3084
status               OpenOpenOpenOpenOpenOpenOpenOpenOpenOpenOpenOp...
reporting_date       2020-06-052020-06-052020-06-052020-06-052020-0...
% deaths                                                       76.8841
num of facilities                                                  554
dtype: object

In [22]:
df2 = pd.read_json(r'../Covid-County-LTC-Analysis/Source_data/IL_2020-05-29LTC_data_Source.json'[FacilityValues])

NameError: name 'FacilityValues' is not defined